In [1]:
import numpy as np
import matplotlib
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import time

import sys
sys.path += ['Package/']  ### Specify the directory where the Package is
from functions_for_DDCSBM import *

import warnings
warnings.filterwarnings("ignore")

## Percolation threshold

In [2]:
n = 1000 # size of the network
T = 15 # number of frames

theta = np.random.uniform(3,10,n)**3 # vector theta (power law degree distribution)
theta = theta/np.mean(theta) # normalization of theta
phi = np.mean(theta**2) # phi
fraction = np.ones(2)/2 # vector pi

n_it = 10 # discretization of the vector c
eta = 0.8 # eta

tau_v = np.linspace(0,1,n_it) # vector tau
c_v = np.linspace(0.01,10,n_it) # vector c

size = np.zeros((n_it,n_it))

In [ ]:
t0 = time.time()
for i in range(n_it): 

    c = c_v[i] # select the value of c
    for j in range(n_it):
        tau = tau_v[j] # select the value of tau

        C = np.array(([c,c],[c,c]))/2 # generate an erdos renyi graph
        AT, Label = adjacency_matrix_series_DDCSBM(T, C, c, eta, theta,fraction) # generate the sequence of adjacency matrices
        Atilde = AT[0].A

        for k in range(1,T):
            mask = np.random.binomial(1,tau,(n,n)) 
            mask = np.tril(mask) + np.tril(mask, -1).T # select the nodes that should be copied
            M = Atilde*mask + AT[k].A*(1-mask) # compute the matrix the new adjacency matrix
            Atilde = ((M - Atilde) == np.ones((n,n)))*1 # compute the matrix Atilde

        G = nx.from_numpy_array(Atilde)
        Gc = max(nx.connected_component_subgraphs(G), key=len) # compute the size of the giant component
        size[i][j] = len(Gc)/n
        
        OUT = 'i : ' + str(i+1) + '; j : ' + str(j+1) 
        sys.stdout.write('\r%s' % OUT)
        sys.stdout.flush()
        
print('\n elapsed time : ' + str(time.time() - t0))

i : 4; j : 60

## (Static) detectability threshold

In [ ]:
c_out = 3
c_in_v = np.linspace(5,15,n_it) # vector to create different values of alpha
tau_v2 = np.linspace(0,1,n_it) # vector tau
detectability = np.zeros((n_it,n_it))

In [ ]:
t0 = time.time()
for i in range(n_it): 

    c_in = c_in_v[i] # select a value of c_in
    c = (c_in+c_out)/2 # compute the average degree
    C = np.array(([c_in, c_out],[c_out, c_in])) # matrix C


    for j in range(n_it):
        tau = tau_v2[j] # select a value of tau

        AT, Label = adjacency_matrix_series_DDCSBM(T, C, c, eta, theta, fraction) # build the sequence of adjacency matrices
        Atilde = AT[0].A

        for k in range(1,T):
            mask = np.random.binomial(1,tau,(n,n))
            mask = np.tril(mask) + np.tril(mask, -1).T # select the edges to be copied
            M = Atilde*mask + AT[k].A*(1-mask) # generate the new adjacency matrix
            Atilde = ((M - Atilde) == np.ones((n,n)))*1 # generate the matrix Atilde
        
        v = 2*Label[-1]-1
        detectability[i][j] = (v@Atilde@v/n)/np.sqrt((np.sum(Atilde)+10**(-8))/n)*np.sqrt(phi) # estimate alpha
            

        OUT = 'i : ' + str(i+1) + '; j : ' + str(j+1) 
        sys.stdout.write('\r%s' % OUT)
        sys.stdout.flush()
        
print('\n elapsed time : ' + str(time.time()-t0)) 

In [ ]:
fig = plt.figure(figsize = (14,12))

ax1 = plt.subplot2grid((4,4), (2,0), colspan=2, rowspan=2)
ax2 = plt.subplot2grid((4,4), (2,2), colspan=2, rowspan=2)


fs = 18
ts = 14

X,Y = np.meshgrid(c_v*phi, tau_v)


im = ax1.pcolormesh(X,Y,size.T, cmap = 'Blues')
ax1.plot(c_v*phi, (c_v*phi-1)/(c_v*phi+10**(-8)), color = 'k', linestyle = '-.')
ax1.set_ylim(0,1)
ax1.set_xlabel(r'$c\Phi$', fontsize = 20)
ax1.set_ylabel(r'$\tau$', fontsize = 20)
ax1.tick_params(axis="x", labelsize=ts)
ax1.tick_params(axis="y", labelsize=ts)
ax1.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')


c_v2 = (c_in_v+c_out)/2
alpha_v2 = (c_in_v - c_out)/np.sqrt(c_v2)/(2/np.sqrt(phi))
X,Y = np.meshgrid(alpha_v2, tau_v2)

beta = 2.5
im2 = plt.pcolormesh(X,Y,np.tanh(beta*(detectability.T-1)), cmap = 'Blues', vmin = 0, vmax = 1)
fig.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04) 
ax2.plot(alpha_v2, 1-1/(alpha_v2+10**(-8))**2, color = 'k', linestyle = '-.')
ax2.set_ylim(0,1)
ax2.set_xlabel(r'$\alpha$', fontsize = 20)
ax2.tick_params(axis="x", labelsize=ts)
ax2.tick_params(axis="y", labelsize=0)
ax2.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')

###################################################################


plt.show();
